In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import LdaModel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import _nmf
from sklearn.preprocessing import normalize
import pickle
from pandas import DataFrame
import requests
import time


In [65]:
#nytimes key
KEY = "xGrPsBfIxHdQnAuxxtGzFcTrmTSJ2uEH"

In [66]:
import datetime

end = datetime.datetime(2022,6,1)

start = datetime.datetime(2017, 4, 1)

ymd_in_range = [x.split(' ') for x in pd.date_range(start, end).strftime("%Y %-m ").tolist()]
ymd_in_range

ymd = [x.pop(0).replace('-','') for x in ymd_in_range]
ymd

['20170401',
 '20170402',
 '20170403',
 '20170404',
 '20170405',
 '20170406',
 '20170407',
 '20170408',
 '20170409',
 '20170410',
 '20170411',
 '20170412',
 '20170413',
 '20170414',
 '20170415',
 '20170416',
 '20170417',
 '20170418',
 '20170419',
 '20170420',
 '20170421',
 '20170422',
 '20170423',
 '20170424',
 '20170425',
 '20170426',
 '20170427',
 '20170428',
 '20170429',
 '20170430',
 '20170501',
 '20170502',
 '20170503',
 '20170504',
 '20170505',
 '20170506',
 '20170507',
 '20170508',
 '20170509',
 '20170510',
 '20170511',
 '20170512',
 '20170513',
 '20170514',
 '20170515',
 '20170516',
 '20170517',
 '20170518',
 '20170519',
 '20170520',
 '20170521',
 '20170522',
 '20170523',
 '20170524',
 '20170525',
 '20170526',
 '20170527',
 '20170528',
 '20170529',
 '20170530',
 '20170531',
 '20170601',
 '20170602',
 '20170603',
 '20170604',
 '20170605',
 '20170606',
 '20170607',
 '20170608',
 '20170609',
 '20170610',
 '20170611',
 '20170612',
 '20170613',
 '20170614',
 '20170615',
 '20170616',

In [73]:
from pandas import DataFrame

q_term = 'bullish'
begin_date = ymd[0]
end_date = ymd[-1]

baseurl = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q="+q_term+"&begin_date="+begin_date+"&end_date="+end_date+"&facet_filter=true&api-key="+KEY
baseurl
response = requests.get(baseurl)
data = response.json()
articles = data["response"]["docs"]
maxPages = 8



for i in range(maxPages):
    response = requests.get(baseurl+ "&page="+str(i))
    data = response.json()
    articles = data["response"]["docs"]
    articles = pd.DataFrame(articles)
    print("Retrieving page ", i)

    if  i == 0:
        pages =  articles
    else:
        pages = pd.concat([pages,articles])
        time.sleep(1) 

Retrieving page  0
Retrieving page  1
Retrieving page  2
Retrieving page  3
Retrieving page  4
Retrieving page  5
Retrieving page  6
Retrieving page  7


In [74]:
pages.to_csv('NYT_topic_bull.csv')

In [91]:
from pandas import DataFrame

q_term = 'bearish'
begin_date = ymd[0]
end_date = ymd[-1]

baseurl = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q="+q_term+"&begin_date="+begin_date+"&end_date="+end_date+"&facet_filter=true&api-key="+KEY
baseurl
response = requests.get(baseurl)
data = response.json()
articles = data["response"]["docs"]
maxPages = 8



for i in range(maxPages):
    response = requests.get(baseurl+ "&page="+str(i))
    data = response.json()
    articles = data["response"]["docs"]
    articles = pd.DataFrame(articles)
    print("Retrieving page ", i)

    if  i == 0:
        pages =  articles
    else:
        pages = pd.concat([pages,articles])
        time.sleep(1) 

Retrieving page  0
Retrieving page  1
Retrieving page  2
Retrieving page  3
Retrieving page  4
Retrieving page  5
Retrieving page  6
Retrieving page  7


In [92]:
pages.to_csv('NYT_topic_bearish.csv')

In [93]:
#pages.columns
#pages.iloc[:,0] #abstract

from nltk.tokenize import word_tokenize
from nltk.text import Text

table_abstract=pages["abstract"].tolist()
table_date = pages["pub_date"].tolist()


table_abs_tokens=[]
for i in range(len(pages)):
    table_abs_tokens.append(word_tokenize(table_abstract[i]))
table_abs_tokens

table_abs_text = []
for i in range(len(pages)):
    table_abs_text.append(Text(table_abs_tokens[i]))
table_abs_text

[<Text: Michael Novogratz , the former Fortress executive ,...>,
 <Text: This week : Jeff Sommer with Sewell Chan...>,
 <Text: The $ 20 million agreement over business practices...>,
 <Text: The activist investor William A. Ackman has unloaded...>,
 <Text: A couple break up , but remain together...>,
 <Text: A documentary about the Syrian civil war focuses...>,
 <Text: Tim Lee , an obscure economist who writes...>,
 <Text: The often irreverent French director François Ozon gets...>,
 <Text: The stock market continued to plunge Monday ,...>,
 <Text: A shakeout in the market seemed long overdue...>,
 <Text: As a candidate , Donald Trump warned about...>,
 <Text: The decade-long rally has faced numerous challenges ....>,
 <Text: Zakiyyah Alexander ’ s ambitious play leaves the...>,
 <Text: The central bank has embraced an easy monetary...>,
 <Text: His songs included “ O Bêbado e a...>,
 <Text: The career of an eclectic musician spanned decades...>,
 <Text: The vow to revamp the system ha

In [94]:
from nltk.corpus import stopwords

stops = stopwords.words('english')
stops.extend('.,[,],(,),;,/,-,\',?,",:,<,>,n\'t,|,#,\'s,\",\'re,\'ve,\'ll,\'d,\'re'.split(','))
stops.extend(',')
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [95]:
stop_removed_token= []
for i in range(len(pages)):
    tokens = [word for word in table_abs_tokens[i] if word not in stops]
    stop_removed_token += [tokens]


In [96]:
data_text = pd.DataFrame(stop_removed_token)

In [97]:
train_headlines = [value[0] for value in data_text.iloc[0:].values]

In [98]:
data_text.iloc[0:].values[3]

array(['The', 'activist', 'investor', 'William', 'A.', 'Ackman',
       'unloaded', 'remaining', 'positions', 'company', '—', 'bet', 'one',
       'time', 'valued', '$', '1', 'billion', 'Pershing', 'Square',
       'Capital', 'Management', None, None, None, None, None, None, None,
       None, None, None], dtype=object)

In [99]:
num_topics = 20

id2word = gensim.corpora.Dictionary(stop_removed_token)
corpus = [id2word.doc2bow(text) for text in stop_removed_token]

lda = LdaModel(corpus = corpus,id2word = id2word, num_topics=num_topics)

In [100]:
def get_lda_topics(model, num_topics):
    word_dict = {}
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20)
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    return pd.DataFrame(word_dict)

In [101]:
get_lda_topics(lda, num_topics)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,fall,The,’,times,’,stronger,master,’,’,top,’,The,In,Securities,’,’,’,advisers,major,looking
1,bitcoin,risks,China,Four,wanting,Apple,As,The,market,bearish,agreement,year,Times,chief,A,would,But,hedge,lagging,With
2,Bloomberg,New,investors,books,Alexander,Pittsburgh,Trump,Amsterdam,stock,The,media,But,museums,Commission,financial,weeks,film,’,reduce,life
3,hedge,But,market,No,works,mailings,The,economy,The,China,executive,economy,critic,false,Jeff,economist,Sunday,multiple,handsomely,biggest
4,said,companies,debt,shares,rates,cash,fragile,easy,But,identified,chief,investor,discussion,public,Bernanke,fallen,musical,Department,quest,big
5,Michael,’,big,Monday,inflation,Flush,economic,football,year,election,final,hearing,ever,making,husband,safe,love,invested,behind,estimates
6,shelving,seemed,sell-off,avoided,Zakiyyah,Google,American,know,investors,crashes,came,government,artist,Exchange,Claire,times,Israeli,law,investors,several
7,know,1990s,worries,different,register,pipe-bomb,president,product,central,Company,day,gloomy,generation,filed,civil,Turkey,fell,fund,recycles,After
8,"8,000",Hill,drops,modern,interest,online,economy,troubles,”,flight,know,alike,two,executive,markets,currency,10,banks,As,His
9,To,years,Liberals,startlingly,leaves,hateful,Street,Shakespeare,He,hedge,Securities,attractive,inspiration,lawsuit,States,writes,getting,Commodities,brick-and-mortar,downward
